In [9]:
import pymongo
import os
import json
import asyncio
import aiohttp

In [10]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
uri = os.environ.get('MONGO_URI')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [11]:
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

In [12]:
client = pymongo.MongoClient(uri)
db = client[os.environ.get('DATABASE')]
collection = db[os.environ.get('COLLECTION')]

/home/neel/anaconda3/lib/python3.9/site-packages/urllib3/poolmanager.py:174: RuntimeWarning: coroutine 'generate_embedding' was never awaited
  self.pools = RecentlyUsedContainer(num_pools, dispose_func=lambda p: p.close())


In [29]:
json.dumps({"query": "My name is neel"})


'{"query": "text"}'

In [32]:
import aiohttp
import json

async def generate_embedding() -> list[float]:
  url = "http://20.193.130.111:5000/preprocess"
  headers = {"Content-Type": "application/json"}
  async with aiohttp.ClientSession() as session:
    async with session.post(url, headers=headers, json=json.loads(json.dumps({"query": "My name is neel"}))) as response:
      json_data = await response.json()
      print(json_data)
      return json_data['embeddings']

query = "flight"
embedding_query = await generate_embedding()


{'embeddings': [-0.11883846670389175, 0.04829873889684677, -0.0025481898337602615, -0.011011169292032719, 0.05195077136158943, 0.010291718877851963, 0.11543328315019608, 0.0007007475942373276, -0.08592537045478821, -0.07065396010875702, 0.0013317846460267901, -0.035472381860017776, 0.0184340737760067, -0.006737224757671356, 0.02440301701426506, -0.029503198340535164, -0.058138418942689896, -0.050439558923244476, -0.020765451714396477, 0.02903594821691513, -0.06367610394954681, 0.024029942229390144, 0.026243260130286217, -0.0060374001041054726, -0.011076578870415688, -0.0014006723649799824, -0.018619786947965622, 0.0327700711786747, 0.002886015921831131, -0.05694399029016495, -0.04394163191318512, 0.025414109230041504, 0.08790943026542664, -0.02499120868742466, -0.03668316453695297, 0.006241375580430031, -0.06646797806024551, -0.0671444684267044, 0.02056426927447319, 0.04238881915807724, 0.02188023552298546, -0.042882438749074936, -0.034377019852399826, 0.06146682798862457, 0.0656372457

In [33]:
embedding_query

[-0.11883846670389175,
 0.04829873889684677,
 -0.0025481898337602615,
 -0.011011169292032719,
 0.05195077136158943,
 0.010291718877851963,
 0.11543328315019608,
 0.0007007475942373276,
 -0.08592537045478821,
 -0.07065396010875702,
 0.0013317846460267901,
 -0.035472381860017776,
 0.0184340737760067,
 -0.006737224757671356,
 0.02440301701426506,
 -0.029503198340535164,
 -0.058138418942689896,
 -0.050439558923244476,
 -0.020765451714396477,
 0.02903594821691513,
 -0.06367610394954681,
 0.024029942229390144,
 0.026243260130286217,
 -0.0060374001041054726,
 -0.011076578870415688,
 -0.0014006723649799824,
 -0.018619786947965622,
 0.0327700711786747,
 0.002886015921831131,
 -0.05694399029016495,
 -0.04394163191318512,
 0.025414109230041504,
 0.08790943026542664,
 -0.02499120868742466,
 -0.03668316453695297,
 0.006241375580430031,
 -0.06646797806024551,
 -0.0671444684267044,
 0.02056426927447319,
 0.04238881915807724,
 0.02188023552298546,
 -0.042882438749074936,
 -0.034377019852399826,
 0.061

In [34]:
results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": embedding_query,
    "path": "embedding",
    "numCandidates": 100,
    "limit": 4,
    "index": "vector_index",
      }}
])

In [35]:
for document in results:
    # print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')
    print(document)

{'_id': ObjectId('66044c0f7f36dfad8a568759'), 'Unnamed: 0': 1, 'Airline Name': 'AB Aviation', 'Overall_Rating': '1', 'Review_Title': '"Not a good airline"', 'Review Date': '25th June 2019', 'Verified': True, 'Review': ' Moroni to Anjouan. It is a very small airline. My ticket advised me to turn up at 0800hrs which I did. There was confusion at this small airport. I was then directed to the office of AB Aviation which was still closed. It opened at 0900hrs and I was told that the flight had been put back to 1300hrs and that they had tried to contact me. This could not be true as they did not have my phone number. I was with a local guide and he had not been informed either. I presume that I was bumped off. The later flight did operate but as usual, there was confusion at check-in. The flight was only 30mins and there were no further problems. Not a good airline but it is the only one for Comoros.', 'Aircraft': 'E120', 'Type Of Traveller': 'Solo Leisure', 'Seat Type': 'Economy Class', 'R